In [1]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm
import file_methods as fm
import input_file as ipf
import generate_inputs as gi
import keyword_block
import sys
import subprocess
import pickle
import pandas as pd
import labels as lbls
import shutil
import yaml

In [11]:
# Import a template CT input file.
template = gi.import_template('CarbSulfModel/CSColumnShortSRAOMMethane.in')

*** Importing template file ***
BaseException: This is normally due to a commented line in the input file. If it is not, something has gone really wrong!
BaseException: This is normally due to a commented line in the input file. If it is not, something has gone really wrong!
The keyword "ION_EXCHANGE" you searched for does not exist. If you are sure that this keyword is in your input file, check your spelling.
The keyword "SURFACE_COMPLEXATION" you searched for does not exist. If you are sure that this keyword is in your input file, check your spelling.
The keyword "PEST" you searched for does not exist. If you are sure that this keyword is in your input file, check your spelling.
The keyword "ISOTOPES" you searched for does not exist. If you are sure that this keyword is in your input file, check your spelling.


In [12]:
dictionary={}
dictionary['concentrations']=dict.fromkeys(template.keyword_blocks['PRIMARY_SPECIES'].contents.keys())
dictionary['reaction']=dict.fromkeys(template.keyword_blocks['AQUEOUS_KINETICS'].contents.keys())
dictionary['transport']=dict.fromkeys(template.keyword_blocks['TRANSPORT'].contents.keys())
dictionary['flow']=dict.fromkeys(template.keyword_blocks['FLOW'].contents.keys())

del dictionary['concentrations']['PRIMARY_SPECIES']
del dictionary['reaction']['AQUEOUS_KINETICS']
del dictionary['transport']['TRANSPORT']
del dictionary['flow']['FLOW']

display(dictionary)

{'concentrations': {'H2(aq)': None,
  'H+': None,
  'Al+++': None,
  'CO2(aq)': None,
  'C13O2(aq)': None,
  'NH3(aq)': None,
  'Mg++': None,
  'Ca++': None,
  'Fe++': None,
  'Fe+++': None,
  'Cl-': None,
  'K+': None,
  'H2S(aq)': None,
  'H2S34(aq)': None,
  'SO4--': None,
  'S34O4--': None,
  'Na+': None,
  'SiO2(aq)': None,
  'O2(aq)': None,
  'B(OH)3(aq)': None,
  'HPO4--': None,
  'NO3-': None,
  'Methane(aq)': None,
  'Methane13(aq)': None,
  'Form': None,
  'Form13': None,
  'C5H7O2NSO4': None},
 'reaction': {'Sulfate_reduction': None,
  '13Sulfate_reduction': None,
  '34Sulfate_reduction': None,
  '3413Sulfate_reduction': None,
  'AOM': None,
  '13AOM': None,
  '34AOM': None,
  '3413AOM': None,
  'S1Methanogenesis': None,
  '13S1Methanogenesis': None,
  'S2Methanogenesis': None,
  '13S2Methanogenesis': None,
  '12->13EqEx': None},
 'transport': {'distance_units': None,
  'time_units': None,
  'calculate_diffusion': None},
 'flow': {'distance_units': None,
  'time_units': None

In [13]:
# Set No. of Runs
dictionary['RunNum']=6

#Pick concentrations you want to set, for list see above.
#Carbon Isotopes
VPDB=0.0112372
#formcomp=np.array([-15,-20,-25,-35,-45,-60])
formcomp=np.linspace(-20,-20,dictionary['RunNum'])
formratios=((formcomp/1000)+1)*VPDB
#co2conc=np.array([0.0015, 0.0020, 0.0025, 0.0030, 0.0035])
co2conc=np.linspace(0.0025,0.0025,dictionary['RunNum'])

co2comp=np.linspace(-0.4,-0.4,dictionary['RunNum'])
co2ratios=((co2comp/1000)+1)*VPDB

dictionary['concentrations']['Form']=1/(formratios+1)
dictionary['concentrations']['Form13']=formratios/(formratios+1)

dictionary['concentrations']['CO2(aq)']=(1/(co2ratios+1))*co2conc
dictionary['concentrations']['C13O2(aq)']=(co2ratios/(co2ratios+1))*co2conc

# Remove any keys you have not set
filtered={k: v for k, v in dictionary['concentrations'].items() if v is not None}
dictionary['concentrations'].clear()
dictionary['concentrations'].update(filtered)  

## Do the same for reaction

#rates=np.array([0,1E8,2E8,3E8,4E8,5E8])
rates=np.linspace(0,0,dictionary['RunNum'])
#fracs=np.array([-35,-40,-45,-50,-55])
fracs=np.linspace(-45,-45,dictionary['RunNum'])
#minors=np.linspace(9741,0,dictionary['RunNum'])

dictionary['reaction']['Aerobic_respiration']=rates
dictionary['reaction']['13Aerobic_respiration']=rates
dictionary['reaction']['Sulfate_reduction']=np.linspace(10000,10000,dictionary['RunNum'])
dictionary['reaction']['13Sulfate_reduction']=dictionary['reaction']['Sulfate_reduction']
dictionary['reaction']['34Sulfate_reduction']=((fracs/1000)+1)*dictionary['reaction']['Sulfate_reduction']
dictionary['reaction']['3413Sulfate_reduction']=((fracs/1000)+1)*dictionary['reaction']['Sulfate_reduction']
dictionary['reaction']['AOM']=np.linspace(150000,150000,dictionary['RunNum']) #5*dictionary['reaction']['Sulfate_reduction']
dictionary['reaction']['13AOM']=dictionary['reaction']['AOM']
dictionary['reaction']['34AOM']=((-45/1000)+1)*dictionary['reaction']['AOM']
dictionary['reaction']['3413AOM']=((-45/1000)+1)*dictionary['reaction']['AOM']
dictionary['reaction']['S1Methanogenesis']=np.linspace(10000,10000,dictionary['RunNum'])
dictionary['reaction']['13S1Methanogenesis']=dictionary['reaction']['S1Methanogenesis']
dictionary['reaction']['S2Methanogenesis']=dictionary['reaction']['S1Methanogenesis']
dictionary['reaction']['13S2Methanogenesis']=((-75/1000)+1)*dictionary['reaction']['S2Methanogenesis']
dictionary['reaction']['12->13EqEx']=np.linspace(1000,1000,dictionary['RunNum'])

# Remove any keys you have not set
filtered={k: v for k, v in dictionary['reaction'].items() if v is not None}
dictionary['reaction'].clear()
dictionary['reaction'].update(filtered)

## and transport

# Remove any keys you have not set
filtered={k: v for k, v in dictionary['transport'].items() if v is not None}
dictionary['transport'].clear()
dictionary['transport'].update(filtered)

## and flow

dictionary['flow']['pump']=np.linspace(0,5E-7,dictionary['RunNum'])

# Remove any keys you have not set
filtered={k: v for k, v in dictionary['flow'].items() if v is not None}
dictionary['flow'].clear()
dictionary['flow'].update(filtered)

#Convert all arrays to lists

for species in dictionary['concentrations'].keys():
    dictionary['concentrations'][species]=dictionary['concentrations'][species].tolist()
for reaction in dictionary['reaction'].keys():
    dictionary['reaction'][reaction]=dictionary['reaction'][reaction].tolist()
for keyword in dictionary['transport'].keys():
    dictionary['transport'][keyword]=dictionary['transport'][keyword].tolist()
for keyword in dictionary['flow'].keys():
    dictionary['flow'][keyword]=dictionary['flow'][keyword].tolist()

display(dictionary)


{'concentrations': {'CO2(aq)': [0.002472230167539263,
   0.002472230167539263,
   0.002472230167539263,
   0.002472230167539263,
   0.002472230167539263,
   0.002472230167539263],
  'C13O2(aq)': [2.776983246073674e-05,
   2.776983246073674e-05,
   2.776983246073674e-05,
   2.776983246073674e-05,
   2.776983246073674e-05,
   2.776983246073674e-05],
  'Form': [0.9891074972077297,
   0.9891074972077297,
   0.9891074972077297,
   0.9891074972077297,
   0.9891074972077297,
   0.9891074972077297],
  'Form13': [0.010892502792270246,
   0.010892502792270246,
   0.010892502792270246,
   0.010892502792270246,
   0.010892502792270246,
   0.010892502792270246]},
 'reaction': {'Sulfate_reduction': [10000.0,
   10000.0,
   10000.0,
   10000.0,
   10000.0,
   10000.0],
  '13Sulfate_reduction': [10000.0,
   10000.0,
   10000.0,
   10000.0,
   10000.0,
   10000.0],
  '34Sulfate_reduction': [9550.0, 9550.0, 9550.0, 9550.0, 9550.0, 9550.0],
  '3413Sulfate_reduction': [9550.0, 9550.0, 9550.0, 9550.0, 9550

In [14]:
#Save and move YAML file
file='FullColumn_Test_8_CheckVel_Run.yaml'
with open(file, 'w') as f:
    data = yaml.dump(dictionary, f)
    
shutil.move(file,'/Users/hjb62/Python/Bateson_Paper/Input_Sheets/'+file)

'/Users/hjb62/Python/Bateson_Paper/Input_Sheets/FullColumn_Test_8_CheckVel_Run.yaml'